In [1]:
import json
import pandas as pd
import numpy as np
import random
import pickle
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['>','!,','.',',']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\elirj\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[(['hello'], 'greetings'), (['hi'], 'greetings'), (['hey'], 'greetings'), (['hola'], 'greetings'), (['good', 'day'], 'greetings'), (['salutations'], 'greetings'), (['Yoo'], 'greetings'), (['I', 'am', 'here', 'to', 'check', 'if', 'I', 'have', 'the', 'virus'], 'diagnosis'), (['I', 'think', 'I', 'have', 'to', 'be', 'diagnosed'], 'diagnosis'), (['Can', 'you', 'diagnose', 'me', '?'], 'diagnosis'), (['I', 'think', 'I', "'m", 'Sick'], 'sick'), (['I', 'might', 'have', 'the', 'virus'], 'sick'), (['I', 'do', "n't", 'feel', 'well'], 'sick'), (['I', 'think', 'I', 'might', 'have', 'been', 'infected'], 'sick'), (['I', 'have', 'received', 'my', 'first', 'and', 'second', 'dose'], 'vaccination'), (['I', 'have', 'the', 'vaccination'], 'vaccination'), (['I', 'am', 'vaccinated', 'including', 'the', 'third', 'booster'], 'vaccination'), (['Yes', 'I', 'have', 'received', 'vaccination'], 'vaccination_positive'), (['yes', ',', 'I', 'have', 'one'], 'vaccination_positive'), (['I', 'have', 'the', 'vaccination'], 

In [2]:

# import json
# intents = json.loads(open('intents.json', 'r+').read())
# print(json.dumps(intents, indent=4))

In [3]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print(words)

["'m", '?', 'Can', 'How', 'I', 'Is', 'Later', 'May', 'No', 'See', 'Sick', 'What', 'Who', 'Yes', 'Yoo', 'age', 'am', 'and', 'are', 'ask', 'be', 'been', 'bleeding', 'bone', 'booster', 'born', 'broken', 'bye', 'campus', 'care', 'check', 'college', 'concussion', 'contageous', 'covid', 'cya', 'day', 'diagnose', 'diagnosed', 'diarrhea', 'do', 'dose', 'enter', 'feel', 'fever', 'first', 'flu', 'good', 'have', 'headache', 'hello', 'here', 'hey', 'hi', 'hola', 'if', 'impact', 'including', 'infected', 'is', 'it', 'me', 'might', 'my', "n't", 'name', 'not', 'old', 'one', 'please', 'possible', 'rash', 'received', 'salutation', 'screen', 'second', 'skin', 'sore', 'swelling', 'take', 'tell', 'the', 'think', 'third', 'this', 'tiredness', 'to', 'vaccinated', 'vaccination', 'virus', 'vomit', 'well', 'were', 'what', 'who', 'will', 'wound', 'year', 'yes', 'you', 'your']


In [4]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))


In [5]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(np.array(train_x), np.array(train_y), epochs=10, batch_size=5,)
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("model.h5", hist)
print("Done")

Epoch 1/10
12/12 [==============================] - 0s 583us/step - loss: 2.6111 - accuracy: 0.0862
Epoch 2/10
12/12 [==============================] - 0s 500us/step - loss: 2.5313 - accuracy: 0.1379
Epoch 3/10
12/12 [==============================] - 0s 500us/step - loss: 2.5488 - accuracy: 0.1034
Epoch 4/10
12/12 [==============================] - 0s 500us/step - loss: 2.4517 - accuracy: 0.1724
Epoch 5/10
12/12 [==============================] - 0s 500us/step - loss: 2.3452 - accuracy: 0.2586
Epoch 6/10
12/12 [==============================] - 0s 500us/step - loss: 2.3137 - accuracy: 0.2414
Epoch 7/10
12/12 [==============================] - 0s 500us/step - loss: 2.2284 - accuracy: 0.3103
Epoch 8/10
12/12 [==============================] - 0s 500us/step - loss: 2.2357 - accuracy: 0.2414
Epoch 9/10
12/12 [==============================] - 0s 417us/step - loss: 2.1132 - accuracy: 0.3103
Epoch 10/10
12/12 [==============================] - 0s 417us/step - loss: 2.0729 - accuracy: 0.3966

12/12 [==============================] - 0s 500us/step - loss: 0.3378 - accuracy: 0.8966
Epoch 72/200
12/12 [==============================] - 0s 584us/step - loss: 0.4181 - accuracy: 0.8793
Epoch 73/200
12/12 [==============================] - 0s 500us/step - loss: 0.4075 - accuracy: 0.8276
Epoch 74/200
12/12 [==============================] - 0s 583us/step - loss: 0.3561 - accuracy: 0.8793
Epoch 75/200
12/12 [==============================] - 0s 583us/step - loss: 0.4487 - accuracy: 0.8621
Epoch 76/200
12/12 [==============================] - 0s 500us/step - loss: 0.3535 - accuracy: 0.8621
Epoch 77/200
12/12 [==============================] - 0s 500us/step - loss: 0.3449 - accuracy: 0.8793
Epoch 78/200
12/12 [==============================] - 0s 500us/step - loss: 0.3632 - accuracy: 0.8448
Epoch 79/200
12/12 [==============================] - 0s 417us/step - loss: 0.2842 - accuracy: 0.8793
Epoch 80/200
12/12 [==============================] - 0s 500us/step - loss: 0.3856 - accuracy: 

Epoch 151/200
12/12 [==============================] - 0s 500us/step - loss: 0.2233 - accuracy: 0.8966
Epoch 152/200
12/12 [==============================] - 0s 500us/step - loss: 0.2103 - accuracy: 0.9310
Epoch 153/200
12/12 [==============================] - 0s 417us/step - loss: 0.2553 - accuracy: 0.8621
Epoch 154/200
12/12 [==============================] - 0s 500us/step - loss: 0.2425 - accuracy: 0.8793
Epoch 155/200
12/12 [==============================] - 0s 500us/step - loss: 0.2630 - accuracy: 0.8621
Epoch 156/200
12/12 [==============================] - 0s 500us/step - loss: 0.2614 - accuracy: 0.8448
Epoch 157/200
12/12 [==============================] - 0s 500us/step - loss: 0.2473 - accuracy: 0.8621
Epoch 158/200
12/12 [==============================] - 0s 500us/step - loss: 0.2117 - accuracy: 0.9138
Epoch 159/200
12/12 [==============================] - 0s 500us/step - loss: 0.2406 - accuracy: 0.9138
Epoch 160/200
12/12 [==============================] - 0s 417us/step - lo